# 학습 데이터 만들기
1. 우리가 학습한 모델을 사용하여 120개 품종에 대해 1차 분류<br>
  -> 각 품종에 대해 happy와 angry로 분류
2. 우리가 분류된 이미지를 다시 한번 정제
  - happy로 분류된 이미지 중에 happy와 거리가 먼 이미지 제거
  - angry로 분류된 이미지 중에 happy인 이미지 제거
  - 이미지를 일일이 볼 필요 없이 스르륵 보면서 맞지 않은 사진만 제거해주세요

## 구글 코랩에 파일 업로드
-> 왼쪽 파일 부분에 마우스 우클릭 해서 업로드<br><br>
업로드 해야 할 파일 목록
- model_weights.pth (학습된 모델의 가중치)
- 각자 image.zip파일 업로드

## 압축 파일 해제
-> 오른쪽 파일 부분에 압축파일이 업로드되는 중에는 파란 동그라미가 표시돼요. 그거 다 끝난 다음에 아래 코드 실행해주세요

In [7]:
import zipfile
zip_file = zipfile.ZipFile('image1.zip') # 각자 zip파일 맞게 image2.zip / image3.zip / ... 이렇게 수정해주세요
zip_file.extractall('image')

## 이미지 파일의 이름들 저장한 리스트

In [2]:
import os
img_list=sorted(os.listdir('image'))

In [3]:
img_list

['Afghan_hound',
 'African_hunting_dog',
 'Airedale',
 'American_Staffordshire_terrier',
 'Appenzeller',
 'Australian_terrier',
 'Bedlington_terrier',
 'Bernese_mountain_dog',
 'Blenheim_spaniel',
 'Border_collie',
 'Border_terrier',
 'Boston_bull',
 'Bouvier_des_Flandres',
 'Brabancon_griffon',
 'affenpinscher',
 'basenji',
 'basset',
 'beagle',
 'black-and-tan_coonhound',
 'bloodhound',
 'bluetick',
 'borzoi',
 'boxer',
 'briard']

## 학습된 모델 사용하는 방법
- 학습된 가중치를 불러온다
- 학습할 때와 동일한 구조로 모델을 설정한다
- 가중치를 load하여 모델을 정의한다

In [4]:
from torchvision import models
import torch.nn as nn
import torch

# 위에 설정했던대로 ResNet18 동일하게 정의
model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

num_classes = 2  # happy와 angry
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 가중치 불러오기
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()  # 평가 모드

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 143MB/s]
<ipython-input-4-705de637875b>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possi

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# 모델 사용해서 이미지 happy, angry로 분류
분류된 결과는 result 폴더 안에 저장된다

In [5]:
import os
import shutil
import torch
from torchvision import models, transforms
from torchvision.models import resnet18
from PIL import Image
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def classify_image(model, image_path):
    try:
        image = Image.open(image_path).convert("RGB")  # 이미지 열기 및 RGB 변환
        input_tensor = transform(image).unsqueeze(0).to(device)  # 배치 차원 추가 및 GPU로 이동
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)  # 예측 클래스
        return predicted.item()  # 0: happy, 1: angry
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

input_folder = "image"  # 상위 폴더
output_folder = "result"  # 결과 저장 폴더
os.makedirs(output_folder, exist_ok=True)  # 결과 폴더 생성

for folder_name in img_list:
    folder_path = os.path.join(input_folder, folder_name)
    if not os.path.isdir(folder_path):  # 폴더인지 확인
        continue

    # 결과 폴더 하위에 해당 폴더 생성
    happy_folder = os.path.join(output_folder, folder_name, "happy")
    angry_folder = os.path.join(output_folder, folder_name, "not_happy")
    os.makedirs(happy_folder, exist_ok=True)
    os.makedirs(angry_folder, exist_ok=True)

    # 해당 폴더의 이미지 분류
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)

        # 이미지 파일만 처리
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue

        # 분류 수행
        result = classify_image(model, image_path)

        if result is not None:
            if result == 0:  # happy
                shutil.copy(image_path, os.path.join(happy_folder, filename))
            elif result == 1:  # angry
                shutil.copy(image_path, os.path.join(angry_folder, filename))

        #print(f"{folder_name}/{filename} -> {'happy' if result == 0 else 'not happy' if result == 1 else 'error'}")

print("모든 이미지 분류 완료!")

모든 이미지 분류 완료!


# 이미지 분류 결과 압축파일로 다운로드

In [6]:
import shutil
from google.colab import files

# 1. result 폴더를 ZIP 파일로 압축
shutil.make_archive('result', 'zip', 'result')  # 'result.zip' 생성

# 2. 로컬 저장소로 다운로드
files.download('result.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>